In [35]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
from datetime import datetime, timedelta
import pandas as pd
import os
%matplotlib widget

In [36]:
sns.set_context('poster', font_scale=1.5)

## Set pyAPES to path from env file

In [37]:
# setting path
import sys
import os
from dotenv import load_dotenv

load_dotenv()
pyAPES_main_folder = os.getenv('pyAPES_main_folder')

sys.path.append(pyAPES_main_folder)

# force iPython re-import modules at each call
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
from pyAPES.utils.utilities import lad_weibul, lad_constant

# Define canopy grid

In [39]:
canopy_grid = np.linspace(0,25,101)

# Define species LAD

In [40]:
pines = lad_weibul(z=canopy_grid, LAI=0.75, h=16, hb=6, species='pine')
pines = pines * 1


In [41]:
spruces = lad_weibul(z=canopy_grid, LAI=0.5, h=6, hb=1.5, species='spruce')
spruces = spruces*2


In [42]:
birch = lad_weibul(z=canopy_grid, h=17, hb=4, LAI=1, species='birch')


In [43]:
field_layer = lad_constant(canopy_grid, 0.25, 0.5, -0.1)


In [44]:
total = pines + birch + spruces + field_layer

In [45]:
np.sum(total*0.25)

3.0

# Set up model parameters

In [49]:
# model forcing: see Ex1_creating_model_forcing.ipynb
forcing_file = '/Users/opa/pyAPES_Opa/forcing/Lettosuo/Lettosuo_forcing_2010_2023.csv'
# # vertical leaf-area density profiles for small and large Black Spruce: see Demo0_ecosystem_structure_US-Prr.ipynb
# lad_file = 'c:/Repositories/pyAPES_main/forcing/US-Prr/BlackSpruce_relative_lad_two_cohorts.csv'

#**************** PARAMETER DICTIONARIES ****************************

gpara = {'dt' : 1800.0,  # timestep in forcing data file [s]
         'start_time' : "2018-06-01",  # start time of simulation [yyyy-mm-dd]
         'end_time' : "2018-09-30",  # end time of simulation [yyyy-mm-dd]
         'forc_filename' : forcing_file, # forcing data file
         'results_directory':'results/'
         }

# --- Model control flags
ctr = {'Eflow': True,  # use ensemble flow statistics; i.e fixed ratio of Utop/ustar.
       'WMA': False,  # assume air-space scalar profiles well-mixed
       'Ebal': True,  # computes leaf temperature by solving energy balance
       'WaterStress': 'Rew',  # How soil water limitations are accounted for: 'Rew' |'PsiL' | None
       'seasonal_LAI': True,  # account for seasonal LAI dynamics
       'pheno_cycle': True,  # account for phenological cycle
       'gm': False # account for finite mesophyll conductance
       }

loc = {'lat': 60.63,  # latitude
       'lon': 23.95  # longitude
       }

# grid
grid = {'zmax': 25.0,  # heigth of grid from ground surface [m]
        'Nlayers': 101  # number of layers in grid [-]
        }


# --- micrometeo ---
micromet = {'zos': 0.01,  # forest floor roughness length [m]  -- not used?
            'dPdx': 0.0,  # horizontal pressure gradient
            'Cd': 0.15,  # drag coefficient
            'Utop': 5.0,  # ensemble U/ustar
            'Ubot': 0.01,  # lower boundary
            'Sc': {'T': 2.0, 'H2O': 2.0, 'CO2': 2.0}  # Schmidt numbers
            }

# --- radiation ---
radiation = {'clump': 0.7,  # clumping index [-]
             'leaf_angle': 1.0,  # leaf-angle distribution [-]
             'Par_alb': 0.1,  # shoot Par-albedo [-]
             'Nir_alb': 0.43,  # shoot NIR-albedo [-]
             'leaf_emi': 0.98  # leaf emissivity [-]
             }

# --- interception ---
interception = {'wmax': 0.35,  # maximum interception storage capacity for rain [kg m-2 per unit of LAI]
                'wmaxsnow': 0.7,  # maximum interception storage capacity for snow [kg m-2 per unit of LAI]
                'w_ini': 0.0,  # initial canopy storage [kg m-2]
                'Tmin': 0.0,  # temperature below which all is snow [degC]
                'Tmax': 2.0,  # temperature above which all is water [degC]
                'leaf_orientation': 0.5, # leaf orientation factor for randomdly oriented leaves
                }

# --- define planttypes ---

z = np.linspace(0, grid['zmax'], grid['Nlayers'])  # grid [m] above ground

pt1 = { 'name': 'pine',
        'LAImax': np.sum(pines*0.25), # maximum annual LAI m2m-2
        'lad': pines/np.sum(pines*0.25),  # leaf-area density m2m-3
        # cycle of photosynthetic activity
        'phenop': {
            'Xo': 0.0,
            'fmin': 0.1,
            'Tbase': -4.67,  # Kolari 2007
            'tau': 8.33,  # Kolari 2007
            'smax': 15.0  # Kolari 2014
            },
        # cycle of LAI
        'laip': {
            'lai_min': 0.8,
            'lai_ini': None,
            'DDsum0': 0.0,
            'Tbase': 5.0,
            'ddo': 45.0,
            'ddmat': 250.0,
            'sdl': 12.0,
            'sdur': 30.0
            },
        # A-gs model
        'photop': {
            'Vcmax': 55.0,
            'Jmax': 108.0,  # 1.97*Vcmax (Kattge and Knorr, 2007)
            'Rd': 1.3,  # 0.023*Vcmax
            'tresp': { # temperature response parameters (Kattge and Knorr, 2007)
                'Vcmax': [72., 200., 649.],
                'Jmax': [50., 200., 646.],
                'Rd': [33.0]
                },
            'alpha': 0.25,   # quantum efficiency parameter -
            'theta': 0.7,   # curvature parameter
            'g1': 2.5,      # stomatal slope kPa^(0.5)
            'g0': 4.0e-3,   # residual conductance mol m-2 s-1
            'kn': 0.5,      # nitrogen attenuation coefficient -
            'beta': 0.95,   # co-limitation parameter -
            'drp': [0.39, 0.83, 0.31, 3.0], # Rew-based drought response
            'photo_model': 'medlyn_farquhar',
            },
        'leafp': {
            'lt': 0.02,     # leaf length scale m
            },
        # root zone
        # root zone: pyAPES.planttype.rootzone.RootUptake
        'rootp': {
            'root_depth': 0.5, # rooting depth [m]
            'beta': 0.943, # root distribution shape parameter [-]
            'root_to_leaf_ratio': 2.0, # fine-root to leaf-area ratio [-]
            'root_radius': 2.0e-3, # [m]
            'root_conductance': 5.0e8, # [s]
            }
        }


pt2 = { 'name': 'spruce',
        'LAImax': np.sum(spruces*0.25), # maximum annual LAI m2m-2
        'lad': spruces/np.sum(spruces*0.25),  # leaf-area density m2m-3
        # cycle of photosynthetic activity
        'phenop': {
            'Xo': 0.0,
            'fmin': 0.1,
            'Tbase': -4.67,  # Kolari 2007
            'tau': 8.33,  # Kolari 2007
            'smax': 15.0  # Kolari 2014
            },
        # cycle of LAI
        'laip': {
            'lai_min': 0.8,
            'lai_ini': None,
            'DDsum0': 0.0,
            'Tbase': 5.0,
            'ddo': 45.0,
            'ddmat': 250.0,
            'sdl': 12.0,
            'sdur': 30.0
            },
        # A-gs model
        'photop': {
            'Vcmax': 60.0,
            'Jmax': 118.0,  # 1.97*Vcmax (Kattge and Knorr, 2007)
            'Rd': 1.4,  # 0.023*Vcmax
            'tresp': { # temperature response parameters (Kattge and Knorr, 2007)
                'Vcmax': [72., 200., 649.],
                'Jmax': [50., 200., 646.],
                'Rd': [33.0]
                },
            'alpha': 0.25,   # quantum efficiency parameter -
            'theta': 0.7,   # curvature parameter
            'g1': 2.5,      # stomatal slope kPa^(0.5)
            'g0': 4.0e-3,   # residual conductance mol m-2 s-1
            'kn': 0.5,      # nitrogen attenuation coefficient -
            'beta': 0.95,   # co-limitation parameter -
            'drp': [0.39, 0.83, 0.31, 3.0], # Rew-based drought response
            'photo_model': 'medlyn_farquhar',
            },
        'leafp': {
            'lt': 0.02,     # leaf length scale m
            },
        # root zone
        # root zone: pyAPES.planttype.rootzone.RootUptake
        'rootp': {
            'root_depth': 0.5, # rooting depth [m]
            'beta': 0.943, # root distribution shape parameter [-]
            'root_to_leaf_ratio': 2.0, # fine-root to leaf-area ratio [-]
            'root_radius': 2.0e-3, # [m]
            'root_conductance': 5.0e8, # [s]
            }
        }

pt3 = { 'name': 'decid',
        'LAImax': np.sum(birch*0.25), # maximum annual LAI m2m-2
        'lad': birch/np.sum(birch*0.25),  # leaf-area density m2m-3
        # cycle of photosynthetic activity
        'phenop': {
            'Xo': 0.0,
            'fmin': 0.01,
            'Tbase': -4.67,  # Kolari 2007
            'tau': 8.33,  # Kolari 2007
            'smax': 15.0  # Kolari 2014
            },
        # annual cycle of LAI
        'laip': {
            'lai_min': 0.1, # relative to LAImax
            'lai_ini': None,
            'DDsum0': 0.0,
            'Tbase': 5.0,
            'ddo': 45.0,
            'ddmat': 250.0,
            'sdl': 12.0,
            'sdur': 30.0
            },
        # A-gs model
        'photop': {
            'Vcmax': 45.0,
            'Jmax': 89.0,  # 1.97*Vcmax (Kattge and Knorr, 2007)
            'Rd': 1.0,  # 0.023*Vcmax
            'tresp': { # temperature response parameters (Kattge and Knorr, 2007)
                'Vcmax': [72., 200., 649.],
                'Jmax': [50., 200., 646.],
                'Rd': [33.0]
                },
            'alpha': 0.25,   # quantum efficiency parameter -
            'theta': 0.7,   # curvature parameter
            'g1': 4.5,      # stomatal slope kPa^(0.5)
            'g0': 1.0e-2,   # residual conductance mol m-2 s-1
            'kn': 0.5,      # nitrogen attenuation coefficient -
            'beta': 0.95,   # co-limitation parameter -
            'drp': [0.39, 0.83, 0.31, 3.0], # Rew-based drought response
            'photo_model': 'medlyn_farquhar',
            },
        'leafp': {
            'lt': 0.05,     # leaf length scale m
            },
        # root zone: pyAPES.planttype.rootzone.RootUptake
        'rootp': {
            'root_depth': 0.5, # rooting depth [m]
            'beta': 0.943, # root distribution shape parameter [-]
            'root_to_leaf_ratio': 2.0, # fine-root to leaf-area ratio [-]
            'root_radius': 2.0e-3, # [m]
            'root_conductance': 5.0e8, # [s]
            }
        }

pt4 = { 'name': 'moss',
        'LAImax': np.sum(field_layer*0.25), # maximum annual LAI m2m-2
        'lad': field_layer/np.sum(field_layer*0.25),  # leaf-area density m2m-3
        # cycle of photosynthetic activity
        'phenop': {
            'Xo': 0.0,
            'fmin': 0.01,
            'Tbase': -4.67,  # Kolari 2007
            'tau': 8.33,  # Kolari 2007
            'smax': 15.0  # Kolari 2014
            },
        # annual cycle of LAI
        'laip': {
            'lai_min': 0.1, # relative to LAImax
            'lai_ini': None,
            'DDsum0': 0.0,
            'Tbase': 5.0,
            'ddo': 45.0,
            'ddmat': 250.0,
            'sdl': 12.0,
            'sdur': 30.0
            },
        # A-gs model
        'photop': {
            'Vcmax': 45.0,
            'Jmax': 89.0,  # 1.97*Vcmax (Kattge and Knorr, 2007)
            'Rd': 1.0,  # 0.023*Vcmax
            'tresp': { # temperature response parameters (Kattge and Knorr, 2007)
                'Vcmax': [72., 200., 649.],
                'Jmax': [50., 200., 646.],
                'Rd': [33.0]
                },
            'alpha': 0.25,   # quantum efficiency parameter -
            'theta': 0.7,   # curvature parameter
            'g1': 4.5,      # stomatal slope kPa^(0.5)
            'g0': 1.0e-2,   # residual conductance mol m-2 s-1
            'kn': 0.0,      # nitrogen attenuation coefficient -
            'beta': 0.95,   # co-limitation parameter -
            'drp': [0.39, 0.83, 0.31, 3.0], # Rew-based drought response
            'photo_model': 'medlyn_farquhar',
            },
        'leafp': {
            'lt': 0.05,     # leaf length scale m
            },
        # root zone
        # root zone: pyAPES.planttype.rootzone.RootUptake
        'rootp': {
            'root_depth': 0.5, # rooting depth [m]
            'beta': 0.943, # root distribution shape parameter [-]
            'root_to_leaf_ratio': 2.0, # fine-root to leaf-area ratio [-]
            'root_radius': 2.0e-3, # [m]
            'root_conductance': 5.0e8, # [s]
            }
        }

""" --- forestfloor --- """

snowpack = {
        'kmelt': 2.31e-5,  # Melting coefficient [kg m-2 s-1 degC-1] (=2.0 mm/C/d)
        'kfreeze': 5.79e-6,  # Freezing  coefficient [kg m-2 s-1 degC-1] (=0.5 mm/C/d)
        'retention': 0.05,  # max fraction of liquid water in snow [-]
        'Tmelt': 0.0,  # temperature when melting starts [degC]
        'optical_properties': {
                'emissivity': 0.97,
                'albedo': {'PAR': 0.8, 'NIR': 0.8}
                },
        'initial_conditions': {'temperature': 0.0,
                               'snow_water_equivalent': 0.0}
        }

soil_respiration = {
        'r10': 2.5, # base rate (bulk heterotrophic + autotrophic) [umol m-2 (ground) s-1]
        'q10': 2.0, # temperature sensitivity [-]
        'moisture_coeff': [0.1, -0.28, 4.325, -3.65], # [minimum_relative_respiration,  p[0], p[1], p[2]]. Moyano et al. 2012 data f = p[0] + p[1]*Sat + p[2]*Sat**2
        #'moisture_coeff': [3.83, 4.43, 1.25, 0.854]  # moisture response; Skopp moisture function param [a ,b, d, g]}
        'beta': 6.0 # expontential decay factor for potential soil respiration. f = np.exp(beta*z). With beta=3.0, ca 80% of respiration is from top 50cm 
        }

# Note: renewed bryophyte parameters

# Based on literature review Pleurozium schreberi and Hylocomium splendens does not differ from each other
# median and range [min, max]
Forest_moss = {
    'name': 'forest mosses',  # Hylocomium splendens and Pleurozium schreberi
    'layer_type': 'bryophyte',
    'coverage': 0.4,
    'height': 0.057,  # range: [0.021, 0.10]
    'roughness_height': 0.01,
    #'dry_mass': 0.668,  # range: [0.484, 1.62]
    'bulk_density': 14.3,  # range: [7.3, 28.74]
    'max_water_content': 9.7,  # range: [7.91, 11.8], fitted value is 27.6
    'water_content_ratio': 0.25,  # max_symplast_water_content:max_water_content -ratio
    #'max_symplast_water_content': 2.4,  # based on fitted value of other mosses than Sphagnum
    'min_water_content': 0.1,
    'porosity': 0.98,

    'photosynthesis': {
        'Vcmax': 10.0, 'Jmax': 17.0, 'Rd': 0.5, # [umol m-2 (ground) s-1] at 25 degC
        'alpha': 0.3, 'theta': 0.8, 'beta': 0.9, # quantum yield [-], curvature [-], co-limitation[-]
        'gref': 0.02, 'wref': 7.0, 'a0': 0.7, 'a1': -0.263, 'CAP_desic': [0.44, 7.0],
        'tresp': { # temperature response 
                'Vcmax': [78., 200., 649.], # [activation energy, deactivation energy, entropy factor [kJ mol-1]]
                'Jmax': [56., 200., 646.],
                'Rd': [33.0]
                },
    },
    'optical_properties': {
        'emissivity': 0.98,
        'albedo': {'PAR': 0.11, 'NIR': 0.29} # albedoes when fully hydrated [-]
    },
    'water_retention': {
        # 'theta_s': 0.526,  # based on fitted value of other mosses than Sphagnum
        # 'theta_r': 0.07,  # based on fitted value of other mosses than Sphagnum
        'alpha': 0.166,  # based on fitted value of other mosses than Sphagnum
        'n': 1.679,  # based on fitted value of other mosses than Sphagnum
        'saturated_conductivity': 1.17e-8,  # [m s-1], based on fitted value of other mosses than Sphagnum
        'pore_connectivity': -2.30, # based on fitted value of other mosses than Sphagnum
    },
    'initial_conditions': {
        'temperature': 10.0,
        'water_content': 10.0
    }

}

Litter = {
    'name': 'Litter',
    'layer_type': 'litter',
    'coverage': 0.6,  # [-]
    'height': 0.03,  # [m]
    'roughness_height': 0.01,  # [m]
    'bulk_density': 45.0,  # [kg m\ :sup:`-3`]
    'max_water_content': 4.0, # 4.0,  # [g g\ :sup:`-1`\ ]
    'water_content_ratio': 0.25,  # max_symplast_water_content:max_water_content -ratio
    #'max_symplast_water_content': 1.0, # [g g\ :sup:`-1`\ ]
    'min_water_content': 0.1,
    'porosity': 0.95,  # [m\ :sup:`3` m\ :sup:`-3`\ ]
    'respiration': {# Taken from baresoil!! per what?
        'q10': 1.6,  # base heterotrophic respiration rate [umolm-2s-1]
        'r10': 2.0,  # temperature sensitivity [-]
        #'moisture_coeff': [add here]
    },
    'optical_properties': {  # [0.1102, 0.2909, 0.98]
        'emissivity': 0.98,  # [-]
        'albedo': {'PAR': 0.11, 'NIR': 0.29} # albedos when fully hydrated [-]
    },
    'water_retention': {#'theta_s': 0.95,  # max_water_content / WATER_DENSITY * bulk_density
                        #'theta_r': 0.01,  # min_water_content /WATER_DENSITY * bulk_density
        'alpha': 0.13,
        'n': 2.17,
        'saturated_conductivity': 1.16e-8,  # [m s-1]
        'pore_connectivity': -2.37,
    },
    'initial_conditions': {
        'temperature': 10.0,
        'water_content': 4.0
    }
}

# --- compile forestfloor parameter dictionary

forestfloor = {
    'bottom_layer_types': {
        'litter': Litter,
        'forest_moss': Forest_moss,
    },
    'snowpack': snowpack,
    'soil_respiration': soil_respiration
}

# --- compile canopy-model parameter dictionary

cpara = {'loc': loc,
         'ctr': ctr,
         'grid': grid,
         'radiation': radiation,
         'micromet': micromet,
         'interception': interception,
         'planttypes': {'pine': pt1, 'spruce': pt2, 'decid': pt3, 'shrubs': pt4},
         'forestfloor': forestfloor
         }

""" --- Soil submodel parameters --- """

# grid and soil properties: pF and conductivity from Launiainen et al. 2015 Hyytiala

soil_grid = {#thickness of computational layers [m]
            'dz': [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
                   0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02,
                   0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
                   0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
            # bottom depth of layers with different characteristics [m]
            'zh': [-0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1., -1.5, -2.0]
            }

soil_properties = {'pF': {  # vanGenuchten water retention parameters
                        'ThetaS': [0.943, 0.882, 0.882, 0.882, 0.882, 0.882, 0.882, 0.882, 0.882, 0.882, 0.882, 0.882],
                        'ThetaR': [0.002, 0.104, 0.104, 0.104, 0.104, 0.104, 0.104, 0.104, 0.104, 0.104, 0.104, 0.104],
                        'alpha': [0.202, 0.044, 0.044, 0.044, 0.044, 0.044, 0.044, 0.044, 0.044, 0.044, 0.044, 0.044],
                        'n': [1.349, 1.349, 1.349, 1.349, 1.349, 1.349, 1.349, 1.349, 1.349, 1.349, 1.349, 1.349]
                        },
                  'saturated_conductivity_vertical': [4.97E-05, 3.21E-05, 2.07E-05, 1.34E-05, 8.63E-06, 5.57E-06,
                                                      3.60E-06, 2.32E-06, 1.50E-06, 9.68E-07, 2.61E-07, 1.16E-07],  # saturated vertical hydraulic conductivity [m s-1]
                  'saturated_conductivity_horizontal': [30*4.97E-05, 20*3.21E-05, 10*2.07E-05, 1.34E-05, 8.63E-06, 5.57E-06,
                                                      3.60E-06, 2.32E-06, 1.50E-06, 9.68E-07, 2.61E-07, 1.16E-07],  # saturated horizontal hydraulic conductivity [m s-1]
                  'solid_heat_capacity': None,  # [J m-3 (solid) K-1] - if None, estimated from organic/mineral composition
                  'solid_composition': {
                         'organic': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
                         'sand': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                         'silt': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                         'clay': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
                         },
                  'freezing_curve': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],  # freezing curve parameter
                  'bedrock': {
                              'solid_heat_capacity': 2.16e6,  # [J m-3 (solid) K-1]
                              'thermal_conductivity': 3.0  # thermal conductivity of non-porous bedrock [W m-1 K-1]
                              }
                  }

# --- water model specs
water_model = {'solve': True,
               'type': 'Richards',  #'Equilibrium', # solution approach 'Equilibrium' for equilibrium approach else solves flow using Richards equation
               'pond_storage_max': 0.0,  #  maximum pond depth [m]
               'initial_condition': {  # (dict) initial conditions
                       'ground_water_level': -0.2,  # groundwater depth [m]
                       'pond_storage': 0.  # initial pond depth at surface [m]
                       },
               'lower_boundary': {  # lower boundary condition (type, value, depth)
                      'type': 'impermeable',
                      'value': None,
                      'depth': -2.0
                       },
               'drainage_equation': {  # drainage equation and drainage parameters
                      'type': 'Hooghoudt',  #
                      'depth': 1.0,  # drain depth [m]
                      'spacing': 45.0,  # drain spacing [m]
                      'width': 1.0,  # drain width [m]
                       }
                }

# --- heat model specs
heat_model = {'solve': True,
              'initial_condition': {
                      'temperature': 4.0,  # initial soil temperature [degC]
                      },
              'lower_boundary': {  # lower boundary condition (type, value)
                      'type': 'temperature',
                      'value': 4.0
                      },
              }

# --- soil model parameter dictionary
spara = {'grid': soil_grid,
         'soil_properties': soil_properties,
         'water_model': water_model,
         'heat_model': heat_model}

In [47]:
# function to read forcing data. See 'forcing/forcing_info.txt' for model forcing variable names and units!
from pyAPES.utils.iotools import read_forcing

# import the multi-layer model (mlm) driver
from pyAPES.pyAPES_MLM import driver

In [50]:
forcing = read_forcing(
    forcing_file=gpara['forc_filename'],
    start_time=gpara['start_time'],
    end_time=gpara['end_time'],
    dt=gpara['dt']
)

In [51]:
params = {
    'general': gpara,   # model configuration
    'canopy': cpara,    # planttype, micromet, canopy, bottomlayer parameters
    'soil': spara,      # soil heat and water flow parameters
    'forcing': forcing  # forging data
}

In [16]:
forcing.columns

Index(['doy', 'Prec', 'P', 'Tair', 'Tdaily', 'U', 'Ustar', 'H2O', 'CO2', 'Zen',
       'LWin', 'diffPar', 'dirPar', 'diffNir', 'dirNir', 'X', 'DDsum'],
      dtype='object')

In [ ]:
if os.path.isfile('/Users/opa/pyAPES_main/Examples/MS_figures/modelled_system/results/lettosuo_test_5.nc'):
    os.remove('/Users/opa/pyAPES_main/Examples/MS_figures/modelled_system/results/lettosuo_test_5.nc')
resultfile, Model = driver(parameters=params,
                           create_ncf=True,
                           result_file= 'lettosuo_test_5.nc'
                          )

INFO pyAPES.pyAPES_MLM driver pyAPES_MLM simulation started. Number of simulations: 1
INFO pyAPES.soil.water __init__ Water balance in soil solved using: RICHARDS EQUATION & HOOGHOUDT
INFO pyAPES.soil.heat __init__ Soil heat balance solved.
INFO pyAPES.canopy.mlm_canopy __init__ Eflow: True, WMA: False, Ebal: True
INFO pyAPES.microclimate.radiation __init__ Shortwave radiation model: ZHAOQUALLS
INFO pyAPES.microclimate.radiation __init__ Longwave radiation model: ZHAOQUALLS
INFO pyAPES.canopy.forestfloor __init__ Forestfloor has 2 bottomlayer types
INFO pyAPES.pyAPES_MLM driver Running simulation number (start time 2025-04-29 13:51): 0
INFO pyAPES.pyAPES_MLM run Running simulation 0


0%.. 10%.. 20%.. 30%.. 40%.. 50%.. 60%.. 70%.. 80%.. 90%.. 

INFO pyAPES.pyAPES_MLM run Finished simulation 0, running time 3054.83 seconds
INFO pyAPES.pyAPES_MLM driver Running time 3054.83 seconds


100%


INFO pyAPES.pyAPES_MLM driver Ready! Results are in: results/lettosuo_test_4.nc


In [52]:
import cProfile, pstats, io
from pstats import SortKey
pr = cProfile.Profile()
pr.enable()
if os.path.isfile('/Users/opa/pyAPES_main/Examples/MS_figures/modelled_system/results/profile_test.nc'):
    os.remove('/Users/opa/pyAPES_main/Examples/MS_figures/modelled_system/results/profile_test.nc')
resultfile, Model = driver(parameters=params,
                           create_ncf=True,
                           result_file= 'profile_test.nc'
                          )
pr.disable()

INFO pyAPES.pyAPES_MLM driver pyAPES_MLM simulation started. Number of simulations: 1
INFO pyAPES.soil.water __init__ Water balance in soil solved using: RICHARDS EQUATION & HOOGHOUDT
INFO pyAPES.soil.heat __init__ Soil heat balance solved.
INFO pyAPES.canopy.mlm_canopy __init__ Eflow: True, WMA: False, Ebal: True
INFO pyAPES.microclimate.radiation __init__ Shortwave radiation model: ZHAOQUALLS
INFO pyAPES.microclimate.radiation __init__ Longwave radiation model: ZHAOQUALLS
/Users/opa/pyAPES_main/pyAPES/bottomlayer/organiclayer.py:1179: RuntimeWarning: invalid value encountered in scalar power
  Psi = -1e-2 / alfa*(s**(1.0 / m) - 1.0)**(1.0 / n)  # m
INFO pyAPES.canopy.forestfloor __init__ Forestfloor has 2 bottomlayer types
INFO pyAPES.pyAPES_MLM driver Running simulation number (start time 2025-05-02 09:22): 0
INFO pyAPES.pyAPES_MLM run Running simulation 0
/Users/opa/pyAPES_main/pyAPES/planttype/planttype.py:614: RuntimeWarning: invalid value encountered in scalar divide
  pt_stats[

0%.. 

/Users/opa/pyAPES_main/pyAPES/bottomlayer/organiclayer.py:1114: RuntimeWarning: invalid value encountered in power
  relative_diffusivity = (np.power(Ta / 293.16, 1.75) * np.power(afp, 10.0/3.0) / porosity**2)


10%.. 20%.. 30%.. 40%.. 50%.. 60%.. 70%.. 80%.. 90%.. 

INFO pyAPES.pyAPES_MLM run Finished simulation 0, running time 1769.22 seconds
INFO pyAPES.pyAPES_MLM driver Running time 1769.23 seconds


100%


INFO pyAPES.pyAPES_MLM driver Ready! Results are in: results/profile_test.nc


In [54]:
s = io.StringIO()
sortby=SortKey.TIME
#sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         318178387 function calls (318021169 primitive calls) in 1787.193 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1225066  287.394    0.000  408.462    0.000 /Users/opa/pyAPES_main/pyAPES/leaf/photo.py:476(photo_c3_medlyn_farquhar)
  2804160  174.831    0.000  413.518    0.000 /Users/opa/pyAPES_main/pyAPES/bottomlayer/organiclayer.py:446(water_heat_tendencies)
  1140264  122.041    0.000  123.793    0.000 /Users/opa/pyAPES_main/pyAPES/leaf/boundarylayer.py:21(leaf_boundary_layer_conductance)
   373888   83.023    0.000  555.503    0.001 /Users/opa/pyAPES_main/pyAPES/planttype/planttype.py:374(leaf_gas_exchange)
  1225066   75.999    0.000   75.999    0.000 /Users/opa/pyAPES_main/pyAPES/leaf/photo.py:913(photo_temperature_response)
    11618   67.216    0.006  132.227    0.011 /Users/opa/pyAPES_main/pyAPES/microclimate/radiation.py:341(canopy_sw_ZhaoQualls)
   186944   63.490    0.000  108.130    0.001 /Users/opa/p